# ClinicalTrials.gov Dataset
## Notebook 1: Data Cleaning

This notebook is for putting together a CSV of all the ClinicalTrials.gov records for modeling and analysis.

The entire dataset was downloaded from ClinicalTrials.gov through their API at [https://clinicaltrials.gov/api/gui](https://clinicaltrials.gov/api/gui). As of February 11, 2020, the total number of records in the database was 329,957. The records were in JSON format, and the files were stored in 427 folders, separated by the first four digits of their record number (example: "NCT0000xxxx").

### Importing libraries

In [1]:
import os, json
import pandas as pd
import numpy as np
import time
pd.set_option('display.max_columns', None)

### Reviewing the data structure

Each folder contains individual records, and studies are grouped by leading part of NCT ID.

In [2]:
# Counting the total number of folders
len(os.listdir('AllAPIJSON/'))

428

In [3]:
# Defining the filepath
filepath = 'AllAPIJSON/'

In [4]:
# Creating a list of files
# Adapted from https://stackoverflow.com/questions/19932130/iterate-through-folders-then-subfolders-and-print-filenames-with-path-to-text-f
def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            r.append(os.path.join(root, name))
    return r

In [5]:
# Checking how many files/records there are total
len(list_files(filepath))

329958

In [6]:
# Renaming this list of files
all_files = list_files(filepath)

In [7]:
# Getting rid of .DS_Store
all_files.remove('AllAPIJSON/.DS_Store')

In [8]:
# Checking that removal worked
len(all_files)

329957

I should expect to have 329,957 trial records.

In [9]:
# Defining a function to get the NCT number from a record
def get_Id(a_json):
    return a_json['FullStudy']['Study']['ProtocolSection']['IdentificationModule']['NCTId']

In [10]:
# Creating a dictionary of all the trial records
t0 = time.time()
dict_of_jsons = {}

for filename in all_files:
    with open(filename) as f_in:
        my_json = json.load(f_in)
        dict_of_jsons[get_Id(my_json)] = my_json
        
print(time.time() - t0)
print((time.time() - t0) / 60)

1832.1208708286285
30.53557995160421


It takes about 30 minutes to assemble all the JSON files.

In [11]:
# Verifying the number of records
len(dict_of_jsons)

329957

I have 329,957 trial records in my dictionary which is exactly what I should have!

In [12]:
# The full record for NCT03266497
dict_of_jsons['NCT03266497']

{'FullStudy': {'Rank': 76870,
  'Study': {'ProtocolSection': {'IdentificationModule': {'NCTId': 'NCT03266497',
     'OrgStudyIdInfo': {'OrgStudyId': 'R01HL115189',
      'OrgStudyIdType': 'U.S. NIH Grant/Contract',
      'OrgStudyIdLink': 'https://projectreporter.nih.gov/reporterapi.cfm?PROJECTNUM=R01HL115189&Fy=all'},
     'Organization': {'OrgFullName': 'Tufts University', 'OrgClass': 'OTHER'},
     'BriefTitle': 'Diet Quality Among U.S.-Born and Foreign-born Non-Hispanic Blacks',
     'OfficialTitle': 'Diet Quality Among U.S.-Born and Foreign-born Non-Hispanic Blacks: NHANES 2003-2012 Data'},
    'StatusModule': {'StatusVerifiedDate': 'August 2017',
     'OverallStatus': 'Completed',
     'ExpandedAccessInfo': {'HasExpandedAccess': 'No'},
     'StartDateStruct': {'StartDate': 'January 2003',
      'StartDateType': 'Actual'},
     'PrimaryCompletionDateStruct': {'PrimaryCompletionDate': 'December 2012',
      'PrimaryCompletionDateType': 'Actual'},
     'CompletionDateStruct': {'Comp

Here I'll create a list of all of the record NCT numbers. It should have 329,957 numbers in it.

In [13]:
record_list = list(dict_of_jsons.keys())
len(record_list)

329957

In [14]:
# Drilling down to a record's official title
dict_of_jsons['NCT03266497']['FullStudy']['Study']['ProtocolSection']['IdentificationModule']['OfficialTitle']

'Diet Quality Among U.S.-Born and Foreign-born Non-Hispanic Blacks: NHANES 2003-2012 Data'

In [15]:
# Drilling down to a record's official title
dict_of_jsons['NCT03266497']['FullStudy']['Study']['ProtocolSection']['StatusModule']['StartDateStruct']['StartDate']

'January 2003'

In [16]:
# Drilling down to a record's NCT number
dict_of_jsons['NCT03266497']['FullStudy']['Study']['ProtocolSection']['IdentificationModule']['NCTId']

'NCT03266497'

In [17]:
# Drilling down to a record's study type
dict_of_jsons['NCT03266497']['FullStudy']['Study']['ProtocolSection']['DesignModule']['StudyType']

'Observational'

In [18]:
# Getting the records' NCT number and initial submission QC approval date
t0 = time.time()

nctid_date = []
for n in record_list:                  
    nct_id = dict_of_jsons[n]['FullStudy']['Study']['ProtocolSection']['IdentificationModule']['NCTId']                     
    sub_day = dict_of_jsons[n]['FullStudy']['Study']['ProtocolSection']['StatusModule']['StudyFirstSubmitQCDate']
    nctid_date.append((nct_id, sub_day))
len(nctid_date) 

print(time.time() - t0)
print((time.time() - t0) / 60)

55.51199007034302
0.9252347032229106


In [19]:
# Verifying the correct number of records
len(nctid_date)

329957

In [20]:
# Creating a dataframe of the NCT IDs and dates
df_nctid_date = pd.DataFrame(nctid_date)
df_nctid_date.head()

,0,1
0,NCT03266497,"August 28, 2017"
1,NCT03262987,"August 24, 2017"
2,NCT03260543,"August 21, 2017"
3,NCT03263416,"August 23, 2017"
4,NCT03266614,"August 28, 2017"


I'm able to get the NCT number and study record approval date from all of the records. 

Unfortunately, there is an issue with the offical title in some of the records. At this point, I'll have to do a little troubleshooting. My strategy will be to pull the titles separately. If the function comes across a title with an issue, it will add a NaN in its place.

In [21]:
# Getting title from the records
# This will add 'NaN' if the trial type is missing
t0 = time.time()

titles = []
for n in record_list:
    try:
        title_id = dict_of_jsons[n]['FullStudy']['Study']['ProtocolSection']['IdentificationModule']['OfficialTitle']
        titles.append(title_id)
    except:
        titles.append(None)
        print('error on ' + n)
        
print(time.time() - t0)
print((time.time() - t0) / 60)

error on NCT03267225
error on NCT03260153
error on NCT03265821
error on NCT01180725
error on NCT01186432
error on NCT01186874
error on NCT01185665
error on NCT01184599
error on NCT01181037
error on NCT01182532
error on NCT01189734
error on NCT01187251
error on NCT01181297
error on NCT01182454
error on NCT01189760
error on NCT01180998
error on NCT01189279
error on NCT01182025
error on NCT01180491
error on NCT01188395
error on NCT01188213
error on NCT01180803
error on NCT01180985
error on NCT01185912
error on NCT01182259
error on NCT01188642
error on NCT01189747
error on NCT01246154
error on NCT01244893
error on NCT01247051
error on NCT01248598
error on NCT01247155
error on NCT01244763
error on NCT01248078
error on NCT01242137
error on NCT01243567
error on NCT01247441
error on NCT01244386
error on NCT01247142
error on NCT01249053
error on NCT01249287
error on NCT01246934
error on NCT01243203
error on NCT01244529
error on NCT01240395
error on NCT01246596
error on NCT01247467
error on NCT0

error on NCT00451373
error on NCT00451061
error on NCT00455130
error on NCT00452712
error on NCT00455468
error on NCT00454428
error on NCT00453752
error on NCT00451347
error on NCT00457704
error on NCT00457509
error on NCT00453570
error on NCT00451672
error on NCT00456534
error on NCT00455949
error on NCT00451438
error on NCT00451984
error on NCT00455247
error on NCT00459966
error on NCT00453557
error on NCT00454103
error on NCT00454545
error on NCT00459459
error on NCT00518193
error on NCT00517231
error on NCT00518869
error on NCT00515918
error on NCT00518804
error on NCT00512603
error on NCT00516841
error on NCT00510757
error on NCT00515554
error on NCT00511940
error on NCT00515684
error on NCT00515606
error on NCT00515775
error on NCT00518700
error on NCT00511290
error on NCT00514436
error on NCT00517426
error on NCT00510588
error on NCT00514748
error on NCT00516061
error on NCT00515944
error on NCT00511706
error on NCT00513201
error on NCT00519909
error on NCT00511251
error on NCT0

error on NCT02929095
error on NCT02920762
error on NCT02920775
error on NCT02920684
error on NCT02926547
error on NCT02926495
error on NCT02921464
error on NCT02928783
error on NCT02924870
error on NCT02929901
error on NCT02927509
error on NCT02927548
error on NCT02923622
error on NCT02925754
error on NCT02920281
error on NCT02925026
error on NCT02929615
error on NCT02929654
error on NCT02923557
error on NCT02924545
error on NCT02923310
error on NCT02927535
error on NCT02927132
error on NCT02920931
error on NCT02922465
error on NCT02799992
error on NCT02792777
error on NCT02798419
error on NCT02798198
error on NCT02796781
error on NCT02798510
error on NCT02792946
error on NCT02793414
error on NCT02793336
error on NCT02797158
error on NCT02791035
error on NCT02795884
error on NCT02790281
error on NCT02790203
error on NCT02794610
error on NCT02794740
error on NCT02796157
error on NCT02799758
error on NCT02799797
error on NCT02796989
error on NCT02795130
error on NCT02793973
error on NCT0

error on NCT00461292
error on NCT00467805
error on NCT00463866
error on NCT00460499
error on NCT00463333
error on NCT00468962
error on NCT00465413
error on NCT00462358
error on NCT00469781
error on NCT00464035
error on NCT00915174
error on NCT00919321
error on NCT00912054
error on NCT00917345
error on NCT00914836
error on NCT00915525
error on NCT00917670
error on NCT00916227
error on NCT00910520
error on NCT00910845
error on NCT00915629
error on NCT00918606
error on NCT00919607
error on NCT00913276
error on NCT00918528
error on NCT00850889
error on NCT00859755
error on NCT00852748
error on NCT00856414
error on NCT00852683
error on NCT00852085
error on NCT00854490
error on NCT00850148
error on NCT00850512
error on NCT00851513
error on NCT00852917
error on NCT00850798
error on NCT00855426
error on NCT00853970
error on NCT00854061
error on NCT00854503
error on NCT00857155
error on NCT00851500
error on NCT00854217
error on NCT00856817
error on NCT00859664
error on NCT00851188
error on NCT0

error on NCT02544698
error on NCT02544620
error on NCT02540317
error on NCT02540213
error on NCT02683382
error on NCT02681848
error on NCT02681536
error on NCT02689297
error on NCT02681445
error on NCT02689414
error on NCT02680496
error on NCT02685085
error on NCT02681991
error on NCT02685839
error on NCT02688972
error on NCT02681952
error on NCT02682914
error on NCT02684214
error on NCT02683343
error on NCT02687750
error on NCT02683226
error on NCT02682524
error on NCT02693639
error on NCT02695667
error on NCT02698813
error on NCT02694237
error on NCT02691234
error on NCT02690311
error on NCT02699658
error on NCT02699398
error on NCT02692612
error on NCT02691182
error on NCT02698930
error on NCT02691273
error on NCT02699593
error on NCT02697500
error on NCT02698358
error on NCT02695394
error on NCT02694120
error on NCT02698007
error on NCT02696577
error on NCT02696213
error on NCT02690168
error on NCT02690324
error on NCT02691364
error on NCT02699580
error on NCT02696863
error on NCT0

...

error on NCT01622322
error on NCT01622270
error on NCT01628237
error on NCT01626911
error on NCT01628276
error on NCT01626287
error on NCT01628744
error on NCT01628601
error on NCT01629342
error on NCT01626625
error on NCT01624415
error on NCT01621841
error on NCT01629134
error on NCT01625117
error on NCT01629524
error on NCT01626716
error on NCT01624025
error on NCT01629251
error on NCT01626521
error on NCT01623921
error on NCT01629602
error on NCT01623258
error on NCT01629550
error on NCT01624987
error on NCT01629225
error on NCT01626703
error on NCT01628588
error on NCT01625247
error on NCT01628614
error on NCT01627236
error on NCT01627522
error on NCT01626768
error on NCT01626482
error on NCT01621074
error on NCT01621035
error on NCT01629238
error on NCT01625702
error on NCT01624129
error on NCT01623479
error on NCT01894295
error on NCT01896206
error on NCT01893788
error on NCT01897636
error on NCT01891370
error on NCT01895036
error on NCT01895348
error on NCT01890941
error on NCT0

error on NCT00703313
error on NCT00705588
error on NCT00702559
error on NCT00705770
error on NCT00708396
error on NCT00704418
error on NCT00700752
error on NCT00703781
error on NCT00703248
error on NCT00702377
error on NCT00702897
error on NCT00706888
error on NCT00703235
error on NCT00708643
error on NCT00705991
error on NCT00584571
error on NCT00584064
error on NCT00584025
error on NCT00583596
error on NCT00585117
error on NCT00584090
error on NCT00586625
error on NCT00586664
error on NCT00583713
error on NCT00584311
error on NCT00583505
error on NCT00580541
error on NCT00586118
error on NCT00588887
error on NCT00585975
error on NCT00588783
error on NCT00581347
error on NCT00581516
error on NCT00583609
error on NCT00588926
error on NCT00583583
error on NCT00589862
error on NCT00589511
error on NCT01100632
error on NCT01105884
error on NCT01103141
error on NCT01101490
error on NCT01100645
error on NCT01109472
error on NCT01102244
error on NCT01102634
error on NCT01103583
error on NCT0

error on NCT00739492
error on NCT00734331
error on NCT00733837
error on NCT00673959
error on NCT00672607
error on NCT00677872
error on NCT00673764
error on NCT00671866
error on NCT00673855
error on NCT00674596
error on NCT00677586
error on NCT00678691
error on NCT00678613
error on NCT00674713
error on NCT00982163
error on NCT00986635
error on NCT00987233
error on NCT00982098
error on NCT00989118
error on NCT00982956
error on NCT00986037
error on NCT00988455
error on NCT00985764
error on NCT00983203
error on NCT00987532
error on NCT00985309
error on NCT00986895
error on NCT00986713
error on NCT00986271
error on NCT01077011
error on NCT01075867
error on NCT01076322
error on NCT01075685
error on NCT01074606
error on NCT01076816
error on NCT01078792
error on NCT01079052
error on NCT01073202
error on NCT01075568
error on NCT01079858
error on NCT01071642
error on NCT01078025
error on NCT01070602
error on NCT01075945
error on NCT01079585
error on NCT01075334
error on NCT01079338
error on NCT0

error on NCT00765349
error on NCT00760487
error on NCT00761202
error on NCT00769665
error on NCT00629785
error on NCT00629252
error on NCT00628459
error on NCT00629161
error on NCT00620854
error on NCT00620737
error on NCT00629668
error on NCT00627302
error on NCT00625274
error on NCT00623363
error on NCT00628173
error on NCT00620893
error on NCT00625222
error on NCT00629733
error on NCT00621933
error on NCT00622037
error on NCT00894530
error on NCT00893308
error on NCT00890318
error on NCT00892762
error on NCT00893932
error on NCT00894218
error on NCT00893139
error on NCT00894517
error on NCT00893698
error on NCT00893620
error on NCT00893009
error on NCT00892970
error on NCT00897780
error on NCT01025167
error on NCT01027156
error on NCT01023841
error on NCT01026116
error on NCT01029535
error on NCT01021579
error on NCT01027546
error on NCT01022528
error on NCT01026766
error on NCT01020149
error on NCT01023152
error on NCT01026350
error on NCT01024855
error on NCT01026649
error on NCT0

error on NCT02771145
error on NCT02778919
error on NCT02770365
error on NCT02773342
error on NCT02771080
error on NCT02776319
error on NCT02770664
error on NCT02778243
error on NCT02770976
error on NCT02777476
error on NCT02771652
error on NCT02773511
error on NCT02776020
error on NCT02773043
error on NCT02777606
error on NCT02772107
error on NCT02779881
error on NCT02880488
error on NCT02886546
error on NCT02886403
error on NCT02886442
error on NCT02881606
error on NCT02888132
error on NCT02888431
error on NCT02887287
error on NCT02882178
error on NCT02886091
error on NCT02887053
error on NCT02881502
error on NCT02886455
error on NCT02889016
error on NCT02882932
error on NCT02884167
error on NCT02881723
error on NCT02882295
error on NCT02880969
error on NCT02880553
error on NCT02881138
error on NCT02889640
error on NCT02883959
error on NCT02887599
error on NCT02883738
error on NCT02885584
error on NCT02882451
error on NCT02885402
error on NCT02884154
error on NCT02887001
error on NCT0

In [22]:
# Making sure all the titles were extracted
len(titles)

329957

In [23]:
# Creating a dataframe with the titles
df_titles = pd.DataFrame(titles)
df_titles.head()

,0
0,Diet Quality Among U.S.-Born and Foreign-born ...
1,Color Doppler U/S vs MSCT Venography in the Di...
2,"A 12-week, Randomized, Double-blind, Placebo-c..."
3,"Regional, Multicentric, Randomized Study Evalu..."
4,"""Recovery 4 US"" - A Photovoice-based Social Me..."


Great, now I have all of the titles, or at least NaNs for any troublesome titles. I'll investigate the reason for some of the missing titles.

For the studies reviewied, it appears that no official title was entered. One such study was NCT02921295. 

https://clinicaltrials.gov/ct2/show/NCT02921295

This was also the case for NCT02442856.

https://clinicaltrials.gov/ct2/show/NCT02442856

The next step is to get all of the study types from the records. I'll use the same code as with the titles.

In [24]:
# Getting study type (interventional, observational) from the records
# This will add 'NaN' if the trial type is missing
t0 = time.time()

study_type = []
for n in record_list:
    try:
        study_class = dict_of_jsons[n]['FullStudy']['Study']['ProtocolSection']['DesignModule']['StudyType']
        study_type.append(study_class)
    except:
        study_type.append(None)
        print('error on ' + n)
        
print(time.time() - t0)
print((time.time() - t0) / 60)

error on NCT03268356
error on NCT03268265
error on NCT03269097
error on NCT03267134
error on NCT03320824
error on NCT03329794
error on NCT03329352
error on NCT03323515
error on NCT01180257
error on NCT01183195
error on NCT01246115
error on NCT01240733
error on NCT01309867
error on NCT01307709
error on NCT01300065
error on NCT01309906
error on NCT00935974
error on NCT00932789
error on NCT00878982
error on NCT00877916
error on NCT00781222
error on NCT00784641
error on NCT00780065
error on NCT00788437
error on NCT00789334
error on NCT00781651
error on NCT00788450
error on NCT00782119
error on NCT00506220
error on NCT00448812
error on NCT02913560
error on NCT02916563
error on NCT02919566
error on NCT02910674
error on NCT02911376
error on NCT02918942
error on NCT02916823
error on NCT02856490
error on NCT02858089
error on NCT02525783
error on NCT02521766
error on NCT02474758
error on NCT02474784
error on NCT02474771
error on NCT02535234
error on NCT02537717
error on NCT02841397
error on NCT0

error on NCT01835769
error on NCT01835366
error on NCT01973621
error on NCT04189913
error on NCT04189081
error on NCT04186806
error on NCT04189224
error on NCT04184973
error on NCT04248868
error on NCT04247581
error on NCT04243239
error on NCT04244357
error on NCT04243369
error on NCT03421145
error on NCT03423251
error on NCT03564366
error on NCT03560141
error on NCT03812679
error on NCT03953274
error on NCT03950895
error on NCT03835026
error on NCT03839199
error on NCT03972995
error on NCT03971721
error on NCT03408444
error on NCT04263532
error on NCT03540576
error on NCT03685136
error on NCT03681886
error on NCT03681496
error on NCT03682432
error on NCT01951131
error on NCT01950767
error on NCT01959178
error on NCT01957982
error on NCT01564589
error on NCT01560156
error on NCT02340832
error on NCT02343068
error on NCT02345811
error on NCT02085174
error on NCT02086812
error on NCT02082847
error on NCT02089646
error on NCT02218450
error on NCT02353377
error on NCT01575210
error on NCT0

error on NCT02746120
error on NCT00767910
error on NCT00766727
error on NCT00764452
error on NCT00764491
error on NCT00760643
error on NCT00769340
error on NCT00769808
error on NCT01022554
error on NCT01029028
error on NCT01166607
error on NCT01164527
error on NCT03003767
error on NCT03142581
error on NCT03147417
error on NCT03148613
error on NCT03147820
error on NCT03147482
error on NCT03142594
error on NCT03143127
error on NCT03024970
error on NCT03021707
error on NCT01000454
error on NCT01008202
error on NCT01140919
error on NCT01148251
error on NCT01147094
error on NCT01145378
error on NCT01143116
error on NCT01145651
error on NCT01286792
error on NCT00748033
error on NCT00747266
error on NCT00748917
error on NCT00608647
error on NCT02761759
error on NCT02764580
error on NCT02624739
error on NCT02622919
error on NCT02626624
error on NCT02634125
error on NCT02634996
error on NCT02634554
error on NCT02631213
error on NCT02638883
error on NCT02771119
error on NCT02888626
error on NCT0

In [25]:
# Verifing the correct number of records
len(study_type)

329957

In [26]:
# Creating a dataframe of the study types
df_study_type = pd.DataFrame(study_type)
df_study_type.head()

,0
0,Observational
1,Observational
2,Interventional
3,Interventional
4,Interventional


One of the records that threw an error was NCT03035318. It was because this study was redacted due to it being a "[Trial of device that is not approved or cleared by the U.S. FDA]."

https://clinicaltrials.gov/ct2/show/NCT03035318

This was the same case for NCT03268265.

https://clinicaltrials.gov/ct2/show/NCT03268265

Now, I can create a dataframe with all of the extracted information and then remove the nulls.

In [27]:
# Merging the first two dataframes together
df_2 = df_nctid_date.merge(df_titles, left_index = True, right_index = True)
df_2.head()

,0_x,1,0_y
0,NCT03266497,"August 28, 2017",Diet Quality Among U.S.-Born and Foreign-born ...
1,NCT03262987,"August 24, 2017",Color Doppler U/S vs MSCT Venography in the Di...
2,NCT03260543,"August 21, 2017","A 12-week, Randomized, Double-blind, Placebo-c..."
3,NCT03263416,"August 23, 2017","Regional, Multicentric, Randomized Study Evalu..."
4,NCT03266614,"August 28, 2017","""Recovery 4 US"" - A Photovoice-based Social Me..."


In [28]:
# Merging the dataframe above with the last dataframe
df = df_2.merge(df_study_type, left_index = True, right_index = True)
df.head()

,0_x,1,0_y,0
0,NCT03266497,"August 28, 2017",Diet Quality Among U.S.-Born and Foreign-born ...,Observational
1,NCT03262987,"August 24, 2017",Color Doppler U/S vs MSCT Venography in the Di...,Observational
2,NCT03260543,"August 21, 2017","A 12-week, Randomized, Double-blind, Placebo-c...",Interventional
3,NCT03263416,"August 23, 2017","Regional, Multicentric, Randomized Study Evalu...",Interventional
4,NCT03266614,"August 28, 2017","""Recovery 4 US"" - A Photovoice-based Social Me...",Interventional


Now that the basic dataframe of all the records has been created, I will clean it up a little before saving it as a final CSV for modeling and analysis. I will reformat the date column, set it as the index, rename the columns, and remove all the NaN rows.

In [29]:
df.isnull().sum()

0_x        0
1          0
0_y    10167
0        849
dtype: int64

There are 10,167 missing titles and 849 missing study types. I will drop these rows.

In [30]:
# Dropping all of the null rows and verifying their removal
df.dropna(inplace = True)
df.isnull().sum()

0_x    0
1      0
0_y    0
0      0
dtype: int64

In [31]:
# Renaming all the columns
df.rename(index = {'study_type': 0, 'date': 1}, columns = {'0_x': 'nct_id', 1: 'date', '0_y': 'title', 0: 'study_type'}, inplace = True)
df.head()

,nct_id,date,title,study_type
0,NCT03266497,"August 28, 2017",Diet Quality Among U.S.-Born and Foreign-born ...,Observational
1,NCT03262987,"August 24, 2017",Color Doppler U/S vs MSCT Venography in the Di...,Observational
2,NCT03260543,"August 21, 2017","A 12-week, Randomized, Double-blind, Placebo-c...",Interventional
3,NCT03263416,"August 23, 2017","Regional, Multicentric, Randomized Study Evalu...",Interventional
4,NCT03266614,"August 28, 2017","""Recovery 4 US"" - A Photovoice-based Social Me...",Interventional


In [32]:
# Changing the date format from string to all numbers
df['date'] = pd.to_datetime(df['date'])
df.head()

,nct_id,date,title,study_type
0,NCT03266497,2017-08-28,Diet Quality Among U.S.-Born and Foreign-born ...,Observational
1,NCT03262987,2017-08-24,Color Doppler U/S vs MSCT Venography in the Di...,Observational
2,NCT03260543,2017-08-21,"A 12-week, Randomized, Double-blind, Placebo-c...",Interventional
3,NCT03263416,2017-08-23,"Regional, Multicentric, Randomized Study Evalu...",Interventional
4,NCT03266614,2017-08-28,"""Recovery 4 US"" - A Photovoice-based Social Me...",Interventional


In [33]:
# Setting the date as the index
df.set_index('date', inplace = True)
df.head()

,nct_id,title,study_type
date,,,
2017-08-28,NCT03266497,Diet Quality Among U.S.-Born and Foreign-born ...,Observational
2017-08-24,NCT03262987,Color Doppler U/S vs MSCT Venography in the Di...,Observational
2017-08-21,NCT03260543,"A 12-week, Randomized, Double-blind, Placebo-c...",Interventional
2017-08-23,NCT03263416,"Regional, Multicentric, Randomized Study Evalu...",Interventional
2017-08-28,NCT03266614,"""Recovery 4 US"" - A Photovoice-based Social Me...",Interventional


In [34]:
# Final CSV export of all of the data
df.to_csv('data/all_records_final.csv')

The data can now be used for EDA. Please see Notebooks 2 and 3.